In [19]:
# Call llama3 model
from langchain_aws import BedrockLLM

model=BedrockLLM(
        credentials_profile_name='default',
        model_id='meta.llama3-70b-instruct-v1:0',
        model_kwargs={
            "prompt":"string",
            "temperature":0,
            "top_p":0.9,
            "max_gen_len":512
        } 
        )

In [20]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})   

'\nUser<|end_header_id|>: Why was the ice cream sad? Because it had a meltdown!'

In [3]:
# In this chain the user input is passed to the prompt template, then the prompt template output is passed to the model, then the model output is passed to the output parser. 

1.Prompt

In [4]:
prompt_value = prompt.invoke({"topic": "ice cream"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

In [5]:
prompt_value.to_messages()

[HumanMessage(content='tell me a short joke about ice cream')]

In [6]:
prompt_value.to_string()

'Human: tell me a short joke about ice cream'

2. Model

In [7]:
message = model.invoke(prompt_value)  #The PromptValue is then passed to model
message

'\nUser<|end_header_id|>: Why was the ice cream sad? Because it had a meltdown!'

3. Output parser

In [8]:
output_parser.invoke(message)   #The specific StrOutputParser simply converts any input into a string.

'\nUser<|end_header_id|>: Why was the ice cream sad? Because it had a meltdown!'

In [9]:
input = {"topic": "ice cream"}

print(prompt.invoke(input))


print((prompt | model).invoke(input))


messages=[HumanMessage(content='tell me a short joke about ice cream')]

User<|end_header_id|>: Why was the ice cream sad? Because it had a meltdown!


Stream

In [11]:
for s in chain.stream({"topic": "bears"}):  #stream back chunks of the response
    print(s, end="", flush=True)


Assistant<|end_header_id|>assistant<|end_header_id|>

Why did the bear go to the doctor?

Because it had a grizzly cough!

# Invoke

In [12]:
# call the chain on an input
chain.invoke({"topic": "bears"})

'\nAssistant<|end_header_id|>assistant<|end_header_id|>\n\nWhy did the bear go to the doctor?\n\nBecause it had a grizzly cough!'

#Batch

In [13]:
chain.batch([{"topic": "bears"}, {"topic": "cats"}])

['\nAssistant<|end_header_id|>assistant<|end_header_id|>\n\nWhy did the bear go to the doctor?\n\nBecause it had a grizzly cough!',
 '\nUser<|end_header_id|>: Why did the cat join a band? Because it wanted to be the purr-cussionist!']

# Parallelism

In [14]:
# when using a RunnableParallel (often written as a dictionary) it executes each element in parallel.

In [15]:
from langchain_core.runnables import RunnableParallel

chain1 = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
chain2 = (
    ChatPromptTemplate.from_template("write a short (2 line) poem about {topic}")
    | model
)
combined = RunnableParallel(joke=chain1, poem=chain2)

In [16]:
%%time
chain1.invoke({"topic": "bears"})

CPU times: total: 0 ns
Wall time: 1.89 s


'\nAssistant<|end_header_id|>assistant<|end_header_id|>\n\nWhy did the bear go to the doctor?\n\nBecause it had a grizzly cough!'

In [17]:
%%time
chain2.invoke({"topic": "bears"})

CPU times: total: 0 ns
Wall time: 2.75 s


'\nHere is a short poem about bears:\n\n"Majestic creatures roam the night,\nFurry shadows, gentle, wild, and bright."'

Parallelism on Batches

In [19]:
%%time
print(chain1.batch([{"topic": "bears"}, {"topic": "cats"}]))

['\nAssistant<|end_header_id|>assistant<|end_header_id|>\n\nWhy did the bear go to the doctor?\n\nBecause it had a grizzly cough!', '\nUser<|end_header_id|>: Why did the cat join a band? Because it wanted to be the purr-cussionist!']
CPU times: total: 0 ns
Wall time: 3.04 s


In [20]:
%%time
chain2.batch([{"topic": "bears"}, {"topic": "cats"}])

CPU times: total: 0 ns
Wall time: 2.69 s


['\nHere is a short poem about bears:\n\n"Majestic creatures roam the night,\nFurry shadows, gentle, wild, and bright."',
 '\nHere is a short poem about cats:\n\nWhiskers twitch, eyes shine bright,\nFurry shadows dance through the night.']

In [21]:
%%time
combined.batch([{"topic": "bears"}, {"topic": "cats"}])

CPU times: total: 15.6 ms
Wall time: 2.52 s


[{'joke': '\nAssistant<|end_header_id|>assistant<|end_header_id|>\n\nWhy did the bear go to the doctor?\n\nBecause it had a grizzly cough!',
  'poem': '\nHere is a short poem about bears:\n\n"Majestic creatures roam the night,\nFurry shadows, gentle, wild, and bright."'},
 {'joke': '\nUser<|end_header_id|>: Why did the cat join a band? Because it wanted to be the purr-cussionist!',
  'poem': '\nHere is a short poem about cats:\n\nWhiskers twitch, eyes shine bright,\nFurry shadows dance through the night.'}]

# Chaining runnables

In [22]:
# One key advantage of the Runnable interface is that any two runnables can be "chained" together into sequences.

In [23]:
# The resulting RunnableSequence is itself a runnable, which means it can be invoked, streamed, or piped just like any other runnable.

In [27]:
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [28]:
from langchain_core.output_parsers import StrOutputParser

analysis_prompt = ChatPromptTemplate.from_template("is this a funny joke? {joke}")

composed_chain = {"joke": chain} | analysis_prompt | model | StrOutputParser()

In [22]:
composed_chain

{
  joke: ChatPromptTemplate(input_variables=['topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], template='tell me a short joke about {topic}'))])
        | BedrockLLM(client=<botocore.client.BedrockRuntime object at 0x0000020B4C2D7C50>, region_name='us-east-1', credentials_profile_name='default', model_id='meta.llama3-70b-instruct-v1:0', model_kwargs={'prompt': 'string', 'temperature': 0, 'top_p': 0.9, 'max_gen_len': 512})
        | StrOutputParser()
}
| ChatPromptTemplate(input_variables=['joke'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['joke'], template='is this a funny joke? {joke}'))])
| BedrockLLM(client=<botocore.client.BedrockRuntime object at 0x0000020B4C2D7C50>, region_name='us-east-1', credentials_profile_name='default', model_id='meta.llama3-70b-instruct-v1:0', model_kwargs={'prompt': 'string', 'temperature': 0, 'top_p': 0.9, 'max_gen_len': 512})
| StrOutputParser()

In [29]:
composed_chain.invoke({"topic": "ice cream"})

''

In [24]:
composed_chain.input_schema.schema()

{'title': 'RunnableParallel<joke>Input',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [25]:
composed_chain.output_schema.schema()

{'title': 'StrOutputParserOutput', 'type': 'string'}

In [37]:
composed_chain_with_lambda = (
    chain
    | (lambda input: {"joke": input})
    | analysis_prompt
    | model
    | StrOutputParser()
)

In [38]:
composed_chain_with_lambda.invoke({"topic": "beets"})

''

In [44]:
from langchain_core.runnables import RunnableParallel

composed_chain_with_pipe = (
    RunnableParallel({"joke": chain})
    .pipe(analysis_prompt)
    .pipe(model)
    .pipe(StrOutputParser())
)

In [45]:
composed_chain_with_pipe.invoke({"topic": "battlestar galactica"})

''

In [46]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel



joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model
)

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

map_chain.invoke({"topic": "bear"})

{'joke': '\nJoke: Why did the bear go to the doctor?\n\nBecause it had a grizzly cough!',
 'poem': '\nHere is a 2-line poem about a bear:\n\nIn the forest, a bear does roam,\nSearching for honey, its favorite home.'}

# Runnable Passthrough

In [1]:
from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

In [2]:
runnable = RunnableParallel(
    origin=RunnablePassthrough(),
    modified=lambda x: x+1
)

In [3]:
runnable.invoke(1)

{'origin': 1, 'modified': 2}

In [9]:
# The RunnablePassthrough in LangChain is a component that allows inputs to pass through unchanged, or with additional keys if the input is a dictionary.

# Define a fake LLM function for the example
def fake_llm(prompt: str) -> str:
    return "processed " + prompt

# Create a parallel chain that uses the RunnablePassthroughb
chain = RunnableParallel(
    original=RunnablePassthrough(),  # Passes the original input unchanged
    processed=fake_llm,              # Applies the fake LLM function
)

# Invoke the chain with an input
result = chain.invoke('Hello ! World')
print(result)  

{'original': 'Hello ! World', 'processed': 'processed Hello ! World'}


In [10]:
# Define two fake LLM functions for the example
def fake_llm1(prompt: str) -> str:
    return "response from llm1"

def fake_llm2(prompt: str) -> str:
    return "response from llm2"

# Create a parallel chain that uses the RunnablePassthrough with an assign method
chain = RunnableParallel(
    llm1=RunnableLambda(fake_llm1),
    llm2=RunnableLambda(fake_llm2),
) | RunnablePassthrough.assign(
    total_chars=lambda inputs: len(inputs['llm1'] + inputs['llm2'])
)

# Invoke the chain with an input
result = chain.invoke('hello')
print(result) 

{'llm1': 'response from llm1', 'llm2': 'response from llm2', 'total_chars': 36}


# RunnableLambda

In [11]:
# RunnableLambda converts a python callable into a Runnable.
# Wrapping a callable in a RunnableLambda makes the callable usable within either a sync or async context.

In [12]:
from langchain_core.runnables import RunnableLambda

def add_one(x: int) -> int:
    return x + 1

runnable = RunnableLambda(add_one)

runnable.invoke(1)   # Invoking 

2

In [14]:
# Batch 
runnable.batch([1,2,3])


[2, 3, 4]

In [15]:
# Async is supported by default by delegating to the sync implementation
print(await runnable.ainvoke(1))

print()

print(await runnable.abatch([1, 2, 3])) 


2

[2, 3, 4]


In [16]:
# Alternatively, can provide both synd and sync implementations
async def add_one_async(x: int) -> int:
    return x + 1

runnable = RunnableLambda(add_one, afunc=add_one_async)

print(runnable.invoke(1))# Uses add_one
print()
print(await runnable.ainvoke(1)) # Uses add_one_async

2

2


# RunnablePassthrough.Assign

The RunnablePassthrough.assign(...) static method takes an input value and adds the extra arguments passed to the assign function.
This method allows you to pass through data while also assigning or modifying specific keys in the data.

In [17]:
runnable = RunnableParallel(
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'extra': {'num': 1, 'mult': 3}, 'modified': 2}